In [256]:
import numpy as np
np.random.seed(42)
simu_times = 50000
simu_type = '追热点'

# Intuition

## 抄底相比于不动：期望收益几乎没什么差距，但抄底拉长来看会导致账户有极大的波动！方差甚至会盖过均值。

## 如果有止损线，很容易会导致离场。而且实际上抄底很容易导致被套。

In [308]:
chaodi_ret =  [1.30, 1.25, 1.20, 1.10, 0.90, 0.80, 0.75, 0.70]
chaodi_prob = [0.05, 0.10, 0.25, 0.30, 0.10, 0.10, 0.05, 0.05]

hot_ret =  [1.20, 1.15, 1.1, 1.00, 0.90, 0.8]
hot_prob = [0.15, 0.15, 0.3, 0.10, 0.15, 0.15]
chaodi_ret = hot_ret
chaodi_prob = hot_prob

N = len(chaodi_ret)
print('抄底胜率%.2f, 抄底一次期望收益%.2f' %(sum(chaodi_prob[i] for i in range(N) if chaodi_ret[i]>1),
                                sum(chaodi_prob[i]*chaodi_ret[i] for i in range(N))-1))
      
chaodi_freq_per_year = 8

jianchi_ret =  [4, 3, 2.5, 2.5, 2.5, 2, 1.8, 1.5, 1.2, 0.5]
jianchi_prob = [0.1] * 10

def assert_func(ret, prob):
    assert len(ret)==len(prob) and abs(sum(prob)-1)<1e-5, 'sum(prob)==%.2f'%sum(prob)

assert_func(chaodi_ret, chaodi_prob)
assert_func(jianchi_ret, jianchi_prob)

抄底胜率0.60, 抄底一次期望收益0.04


In [309]:
for year in [1, 2, 3, 4, 5][:]:
    n_stocks_jianchi = 8
    n_stocks_test = 2
    np.random.seed(42)
    years = 3 * year
    chaodi_times = chaodi_freq_per_year * years
    chaodi = np.random.choice(chaodi_ret, size=(simu_times,n_stocks_test, chaodi_times), p=chaodi_prob)
   
    jianchi = np.random.choice(jianchi_ret, size=(simu_times,n_stocks_jianchi, 1), p=jianchi_prob)
    jianchi = np.tile(jianchi, (1, year))

    print('不%s, %s年    '%(simu_type, years), calcu_mu_std(jianchi))
    print('  %s, %s年    '%(simu_type, years), calcu_mu_std(chaodi))
    print()

不追热点, 3年     均值2.15, 方差0.33
  追热点, 3年     均值2.42, 方差1.22

不追热点, 6年     均值5.49, 方差1.51
  追热点, 6年     均值5.88, 方差4.66

不追热点, 9年     均值15.72, 方差6.33
  追热点, 9年     均值14.20, 方差15.24

不追热点, 12年     均值48.89, 方差25.88
  追热点, 12年     均值34.35, 方差48.42

不追热点, 15年     均值161.99, 方差104.65
  追热点, 15年     均值83.31, 方差158.11



In [234]:
def calcu_mu_std(X, leverage=1, zhisun=0):
    dims = X.ndim
    ## 当账户加杠杆，亏完本金则止损
    if leverage > 1:
        zhisun = leverage - 1
        
    X = X.cumprod(axis=-1) * leverage
    if dims == 2:
        X = X[:,np.newaxis,:]
    a,b,c = X.shape
    X = X.reshape((-1,X.shape[-1]))
    condi = (X > zhisun)
    tmp = condi.all(axis=-1)
    rets = tmp * X[np.arange(X.shape[0]),-1] + (1-tmp) * X[np.arange(X.shape[0]),condi.argmax(axis=-1)]
    rets = np.mean(rets.reshape((a,b)), axis=1)
    
    res = '均值%.2f, 方差%.2f'%(np.mean(rets) - (leverage-1), np.std(rets))
    if zhisun > 0:
        res = res + ', 止损线%.2f' %zhisun
    return res